# Построение вложенного словаря
В этом блоке нам понадобятся данные о визитах и покупках, сгруппированные по источнику и каналу.

Данные по визитам (visits_by_source_and_medium.txt)

Данные по покупкам (orders_by_source_and_medium.txt)

Поиск по одному столбцу - ситуация не самая часто встречающаяся. Обычно столбцов гораздо больше. Да и файлы, с которыми приходится работать на практике гораздо сложнее. Как "масштабировать" наше решение, чтобы оно могло учитывать любое количество столбцов и при этом не теряло в скорости?

Для удобства представим более реальную таблицу с покупками. Например, на 4 миллиона строк и 500 Мб на диске. Если необходимо найти в этой таблице нужную комбинацию source и medium, то надо учитывать, что эта пара может встретиться в любой из 4 миллионов строк. Также может быть ситуация, что для этой комбинации source и medium не было ни одной покупки.

Для ускорения нашего поиска давайте заменим таблицу из столбцов source, medium и количества покупок вложенным словарем. Пример такого преобразования:

Соответствующий словарь

In [2]:
orders_dict = {
    'google': {
        'sem': 56,
        'seo': 15
    },
  
    'newsletter': {
        'email': 5
    }
}

Т. е. первичными ключами словаря будут все значения переменной source. Для каждого ключа словаря в качестве значений создаем еще один словарь. Ключами которого уже будут значения столбца medium. И в качестве значений ставим количество покупок orders.

# Что дает такая структура?
Теперь, чтобы найти нужную комбинацию значений source и medium нужно сделать два простых шага:

1. Из 2000 значений source определить есть ли среди них нужный нам. Если его не оказалось, то сразу возвращаем 0 покупок.

2. Если значение source есть в первичных ключах, то ищем значение medium в списке из вторичных ключей, которые оказались у источника source (кстати их может быть всего несколько). Логика такая же - если ключ найден, то возвращаем значение покупок для первичного ключа source и вторичного medium. Если не найден, то возвращаем 0

Итого для поиска среди 4 000 000 строк нам нужно сначала проверить наличие очередного значения в списке ключей из  source, а затем - в списке из medium. Это намного быстрее, чем любой последовательный мониторинг этих 4 миллионов строк.

Давайте реализуем этот алгоритм в коде. Сначала преобразуем знакомый нам словарь orders_dict, чтобы из таблицы со столбцами source и medium получать словарь с двумя уровнями ключей.

# Метод setdefault
Для более наглядной записи используем метод setdefault, который позволяет не проводить проверку наличия ключа в словаре. Чтобы прибавить к значению словаря число мы использовали следующую проверку:

In [3]:
orders_dict = {}

if 'google' in orders_dict:

    orders_dict['google'] += 1

else:

    orders_dict['google'] = 1

print(orders_dict)

{'google': 1}


Можно переписать этот код с помощью метода setdefault. Этот метод проверяет есть ли в словаре указанный ключ 'google'. Если есть, то оставляет соответствующее значение ключа прежним. Если ключа не оказалось, то подставляет указанное нами значение (в примере это значение 0). Тем самым после применения метода setdefault можно смело использовать прибавление 1 к ключу 'google'. Независимо от того был ли этот ключ в словаре раньше:

In [4]:
orders_dict = {}

orders_dict.setdefault('google', 0)

orders_dict['google'] += 1

In [5]:
orders_dict = {}

with open('./module10_files/orders_by_source_and_medium.txt', 'r') as f:
    for line in f:
        line = line.strip().split('\t')
        source = line[0]
        medium = line[1]
        
        orders_count = int(line[2])
        orders_dict.setdefault(source, {})
        orders_dict[source].setdefault(medium, 0)
        orders_dict[source][medium] = orders_count

In [12]:
orders_dict['google']

KeyError: 'google1'

# Домашнее задание
(1 возможный балл)
Измените код функции searchForLine из прошлого блока, чтобы по названию источника source и канала medium получать соответствующее количество покупок из файла orders_by_source_and_medium.txt. Соответственно, функция должна "пройти" следующие тесты:

In [19]:
def searchForLine(source, medium, orders_dict):
    """
    Функция по названию источника source ищет соответствующую строку в файле orders_by_source.txt.
    Возвращает количество покупок, соответствующее источнику source. Если источник не найден, то возвращает 0   

    Пример

    searchForLine('burgerclub')
    1197
   
    searchForLine('source_123')
    0
    """
    if source in orders_dict:
        if medium in orders_dict[source]:
            return int(orders_dict[source][medium])
    return 0

In [20]:
assert searchForLine('google', 'seo', orders_dict) == 15
assert searchForLine('google_123', 'seo', orders_dict) == 0
assert searchForLine('google', 'seo_123', orders_dict) == 0

In [21]:
searchForLine('google', 'sem', orders_dict)

56